In [80]:
import numpy as np
import statsmodels.api as sm

import sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score

from load_data import DataLoader
from HDP_GLM_Simex import VerticalGLMs

In [113]:
family = sm.families.Binomial()

# both reproduction attempts use a infinite privacy budget i.e. no noise added

# this tries to reproduce exactly the estimation process for regularized logistic regression in the paper
split_model = VerticalGLMs("breast", family)

# this instead tries to use the SGD in sklearn to see if it works better
split_model_2 = SGDClassifier(max_iter=1, alpha=0.001, eta0=0.1, learning_rate='constant', 
                              random_state=42, average=True) 

def fit_split_model_2():
    for x in range(10):
        split_model_2.partial_fit(split_model.data.X_train, split_model.data.y_train, classes=[0,1])
        split_model_2.coef_[0][:15] /= max(1, np.sqrt(np.sum(split_model_2.coef_[0][:15] **2)))
        split_model_2.coef_[0][15:] /= max(1, np.sqrt(np.sum(split_model_2.coef_[0][15:] **2)))
    return split_model_2.coef_[0]    

# global model
model = sm.GLM(split_model.data.y_train, split_model.data.X_train, family = split_model.family)

In [114]:
global_params = model.fit_regularized(alpha=0.001, L1_wt=0.0, maxiter=10).params
split_params = split_model.fit(max_epochs = 10)
split2_params = fit_split_model_2()

/home/bob/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bob/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bob/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bob/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

In [115]:
y_pred_global = [0 if x < 0.5 else 1 for x in model.predict(global_params)]
y_pred_split = [0 if x < 0.5 else 1 for x in model.predict(split_params)]
y_pred_split2 = [0 if x < 0.5 else 1 for x in model.predict(split2_params)]

In [116]:
for y_pred in [y_pred_global, y_pred_split, y_pred_split2]:
    print(accuracy_score(sklearn.utils.validation.column_or_1d(split_model.data.y_train), y_pred))

0.8506151142355008
0.6274165202108963
0.9086115992970123


In [119]:
# crucially, the SGD from sklearn works much better so this needs actually further investigation